# Power System Planning Project

## 1. Generation Expansion and Long-Term Storage Planning

### 1.1. Mathematical Formulation

$\begin{align}
\min \quad &\sum_{g\in G} C^{G,F} CAP_g + \sum_{r\in R} C^{R,F} CAP_r + \sum_{s\in S} (C^{S,P} CAP_s + C^{S,E} SOCmax_s) + \\
&\sum_{t\in T} \left(\sum_{g\in G} C^{G,V} GEN_{g,t} + C^{G,S} START_{g,t} + \sum_{r\in R} C^{R,V} REN_{r,t}+VoLL\times NSE_t\right)\\
\text{s.t. } \quad &\sum_{g\in G} GEN_{g,t} + \sum_{r\in R} REN_{r,t} + \sum_{s\in S} \left(DISCHARGE_{s,t} - CHARGE_{s,t} \right) + NSE_t = L_t &\forall t\in T \\
&0 \leq GEN_{g,t} \leq CAP_g \times COMMIT_{g,t} &\forall g\in G, \forall t\in T \\
&COMMIT_{g,t} >= \sum^{t}_{t'\geq t - MinUp_g} START_{g,t} &\forall g\in G, \forall t\in T \\
&1 - COMMIT_{g,t} >= \sum^{t}_{t'\geq t - MinDown_g} SHUT_{g,t} &\forall g\in G, \forall t\in T \\
&COMMIT_{g,t+1} - COMMIT_{g,t} = START_{g,t+1} - SHUT_{g,t+1} &\forall g\in G, t = 1..T-1\\
&GEN_{g,t+1} - GEN_{g,t} \leq RampUp_g &\forall g\in G, t = 1..T-1 \\
&GEN_{g,t} - GEN_{g,t+1} \leq RampDown_g &\forall g\in G, t = 1..T-1 \\
&0 \leq REN_{r,t}    \leq CAP_r \times cf_{r,t} &\forall r\in R, \forall t\in T \\
&REN_{r,t+1} - REN_{r,t} \leq RampUp_r &\forall r\in R, t = 1..T-1 \\
&REN_{r,t} - REN_{r,t+1} \leq RampDown_r &\forall r\in R, t = 1..T-1 \\
&0 \leq CHARGE_{s,t} \leq CAP_s &\forall s\in S, \forall t\in T \\
&0 \leq DISCHARGE_{s,t} \leq CAP_s &\forall s\in S, \forall t\in T \\
&0 \leq SOC_{s,t} \leq SOCmax_s &\forall s\in S, \forall t\in T \\
&SOC_{s,t+1} = SOC_{s,t} + CHARGE_{s,t+1} \times Eff_s - DISCHARGE_{s,t+1}/Eff_s &\forall s\in S, t = 1..T-1 \\
&SOC_{s,min(T)} = SOC_{s,max(T)} = 0.5 \times SOCmax_s &\forall s\in S \\
&NSE_t \geq 0 &\forall t\in T \\
\end{align}$

### 1.2. Modelling with JuMP

In [1]:
using JuMP, HiGHS
using DataFrames, CSV, XLSX

stor_info = DataFrame(CSV.File("data/sample/stor_info.csv"))
ther_info = DataFrame(CSV.File("data/sample/ther_info.csv"))
ren_info = DataFrame(CSV.File("data/sample/ren_info.csv"))
load_forecast = DataFrame(CSV.File("data/sample/load_forecast.csv"))
capacity_factor = DataFrame(CSV.File("data/sample/capacity_factor.csv"))

stor_info = stor_info[:,[:id, :name, :power_cost, :energy_cost, :eff]]
ther_info = ther_info[:,[:id, :name, :fixed_cost, :var_cost, :start_up_cost, :ramp_up_rate, :ramp_dn_rate, :min_up_time, :min_dn_time]]
ren_info = ren_info[:,[:id, :name, :fixed_cost, :var_cost, :ramp_up_rate, :ramp_dn_rate]]
load_forecast = load_forecast[:,[:hour, :load]]
capacity_factor = capacity_factor[:,[:hour, :r_id, :cf]];


In [2]:
# stor_info = id, name, power_cost, energy_cost, eff
# ther_info = id, name, fixed_cost, var_cost, start_up_cost, ramp_up_rate, ramp_dn_rate, min_up_time, min_dn_time
# ren_info = id, name, fixed_cost, var_cost, ramp_up_rate, ramp_dn_rate
# load_forecast = hour, load
# capacity_factor = hour, r_id, cf

function GESP(stor_info, ther_info, ren_info, load_forecast, capacity_factor, VoLL=30000, bigM = 10000, mip_gap=0.01)

    # SETS
    G = ther_info.id
    R = ren_info.id
    S = stor_info.id
    T = load_forecast.hour
    T1 = T[2:end]

    # INIT
    model = Model()
    set_optimizer(model, HiGHS.Optimizer)
    set_optimizer_attribute(model, "mip_rel_gap", mip_gap)

    # VARIABLES
    @variables(model, begin
        CAPG[G] >= 0
        CAPG_COMMIT[G,T] >= 0
        CAPR[R] >= 0
        CAPS[S] >= 0
        SOCM[S] >= 0
        GEN[G,T] >= 0
        REN[R,T] >= 0
        CHARGE[S,T] >= 0
        DISCHARGE[S,T] >= 0
        SOC[S,T] >= 0
        COMMIT[G,T], Bin
        START[G,T], Bin
        SHUT[G,T], Bin
        NSE[T] >= 0
    end)

    # INVESTMENT COSTS
    @expression(model, ThermalIC, sum(ther_info[ther_info.id .== g, :fixed_cost][1] * CAPG[g] for g in G))
    @expression(model, RenewableIC, sum(ren_info[ren_info.id .== r, :fixed_cost][1] * CAPR[r] for r in R))
    @expression(model, StoragePowIC, sum(stor_info[stor_info.id .== s, :power_cost][1] * CAPS[s] for s in S))
    @expression(model, StorageEnIC, sum(stor_info[stor_info.id .== s, :energy_cost][1] * SOCM[s] for s in S))
    @expression(model, TIC, ThermalIC + RenewableIC + StoragePowIC + StorageEnIC)

    # OPERATION COSTS
    @expression(model, ThermalOC, sum(ther_info[ther_info.id .== g, :var_cost][1] * GEN[g,t] for g in G for t in T))
    @expression(model, RenewableOC, sum(ren_info[ren_info.id .== r, :var_cost][1] * REN[r,t] for r in R for t in T))
    @expression(model, StartupOC, sum(ther_info[ther_info.id .== g, :start_up_cost][1] * START[g,t] for g in G for t in T))
    @expression(model, NSE_C, sum(VoLL * NSE[t] for t in T))
    @expression(model, TOC, ThermalOC + RenewableOC + StartupOC + NSE_C)

    # OBJECTIVE FUNCTION
    @objective(model, Min, TIC + TOC)

    # POWER BALANCE CONSTRAINT
    @constraint(model, PowerBalanceC[t in T], sum(GEN[g,t] for g in G) + sum(REN[r,t] for r in R) + sum(DISCHARGE[s,t] for s in S) - sum(CHARGE[s,t] for s in S) + NSE[t] == load_forecast[load_forecast.hour.==t,:load][1])

    # THERMAL GENERATION LIMIT CONSTRAINT
    @constraint(model, LessThanMC[g in G, t in T], CAPG_COMMIT[g,t] <= bigM * COMMIT[g,t])
    @constraint(model, LessThanCapC[g in G, t in T], CAPG_COMMIT[g,t] <= CAPG[g])
    @constraint(model, AuxIsRealC[g in G, t in T], CAPG_COMMIT[g,t] >= CAPG[g] - (1- COMMIT[g,t]) * bigM)
    @constraint(model, ThermalLimC[g in G, t in T], GEN[g,t] <= CAPG_COMMIT[g,t])

    # START CONSTRAINT
    @constraint(model, StartC[g in G, t in T], 
        COMMIT[g,t] >= sum(START[g,i] for i in max(1,t-ther_info[ther_info.id.==g,:min_up_time][1]):t))
        
    @constraint(model, cAvoidCommitBeforeStart[g in G, t in T],COMMIT[g,t] <= sum(START[g,t] for t in minimum(T):t))

    # SHUT CONSTRAINT
    @constraint(model, ShutC[g in G, t in T], 
        1 - COMMIT[g,t] >= sum(SHUT[g,i] for i in max(1,t-ther_info[ther_info.id.==g,:min_dn_time][1]):t))

    # COMMIT CONSTRAINT
    @constraint(model, CommitC[g in G, t in T1], COMMIT[g,t] - COMMIT[g,t-1] == START[g,t] - SHUT[g,t])

    # THERMAL RAMP UP CONSTRAINT
    @constraint(model, TherRampUpC[g in G, t in T1], 
        GEN[g,t] - GEN[g,t-1] <= ther_info[ther_info.id.==g,:ramp_up_rate][1] * CAPG[g])
    
    # THERMAL RAMP DOWN CONSTRAINT
    @constraint(model, TherRampDnC[g in G, t in T1], 
        GEN[g,t-1] - GEN[g,t] <= ther_info[ther_info.id.==g,:ramp_dn_rate][1] * CAPG[g])

    # VRE LIMIT CONSTRAINT
    @constraint(model, VreC[r in R, t in T], 
        # REN[r,t] <= CAPR[r] * capacity_factor[(capacity_factor.r_id.==r) .& (capacity_factor.hour.==t), :cf][1])
        REN[r,t] <= 500 * capacity_factor[(capacity_factor.r_id.==r) .& (capacity_factor.hour.==t), :cf][1])
    
    # VRE RAMP UP CONSTRAINT
    @constraint(model, VreRampUpC[r in R, t in T1], 
        REN[r,t] - REN[r,t-1] <= ren_info[ren_info.id.==r,:ramp_up_rate][1] * CAPR[r])
    
    # VRE RAMP DOWN CONSTRAINT
    @constraint(model, VreRampDnC[r in R, t in T1], 
        REN[r,t-1] - REN[r,t] <= ren_info[ren_info.id.==r,:ramp_dn_rate][1] * CAPR[r])
    
    # CHARGE LIMIT CONSTRAINT
    @constraint(model, ChargeC[s in S, t in T], CHARGE[s,t] <= CAPS[s])

    # DISCHARGE LIMIT CONSTRAINT
    @constraint(model, DischargeC[s in S, t in T], DISCHARGE[s,t] <= CAPS[s])

    # SOC LIMIT CONSTRAINT
    @constraint(model, SocLimitC[s in S, t in T], SOC[s,t] <= SOCM[s])

    # SOC UPDATE CONSTRAINT
    @constraint(model, SocUpdateC[s in S, t in T1],
        SOC[s, t] == SOC[s,t-1] + (CHARGE[s,t]*stor_info[stor_info.id.==s,:eff][1]) - (DISCHARGE[s,t]/stor_info[stor_info.id.==s,:eff][1]))
    
    # SOC INI CONSTRAINT
    @constraint(model, SocIniC[s in S], SOC[s,1] == 0.5 * SOCM[s])

    # SOC END CONSTRAINT
    @constraint(model, SocEndC[s in S], SOC[s,length(T)] == 0.5 * SOCM[s])

    # SOLVE GESP
    optimize!(model)

    return(
        CAPG = value.(CAPG).data,
        CAPG_COMMIT = value.(CAPG_COMMIT).data,
        CAPR = value.(CAPR).data,
        CAPS = value.(CAPS).data,
        SOCM = value.(SOCM).data,
        GEN = value.(GEN).data,
        REn = value.(REN).data,
        CHARGE = value.(CHARGE).data,
        DISCHARGE = value.(DISCHARGE).data,
        SOC = value.(SOC).data,
        COMMIT = value.(COMMIT).data,
        START = value.(START).data,
        SHUT = value.(SHUT).data,
        NSE = value.(NSE).data,
        ThermalIC = value(ThermalIC),
        StoragePowIC = value(StoragePowIC),
        StorageEnIC = value(StorageEnIC),
        ThermalOC = value(ThermalOC),
        RenewableOC = value(RenewableOC),
        StartupOC = value(StartupOC),
        NSE_C = value(NSE_C),
        TIC = value(TIC),
        TOC = value(TOC),
        OV = objective_value(model)
    )
end
    



GESP (generic function with 4 methods)

In [3]:
sol = GESP(stor_info, ther_info, ren_info, load_forecast, capacity_factor,30000, 10000, 0.01)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
303 rows, 240 cols, 903 nonzeros
263 rows, 212 cols, 871 nonzeros

Solving MIP model with:
   263 rows
   212 cols (19 binary, 0 integer, 0 implied int., 193 continuous)
   871 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -inf            inf                  inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   754952.475823   77122094.37582    99.02%        0      0      0       153     0.0s
 L       0       0         0   0.00%   754952.900311   754952.900311      0.00%       30      5      0       162     0.0s

Solving report
  Status            Optimal
  Primal bound      754952.900311
  Dual bound        754952.900311
  G

(CAPG = [0.0, -0.0, 1510.2377853529313], CAPG_COMMIT = [-0.0 -0.0 -0.0 -0.0; 0.0 0.0 0.0 -0.0; 1510.2377853529313 1510.2377853529313 1510.2377853529313 1510.2377853529313], CAPR = [0.0, 825.730113], CAPS = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 526.3283763017587, 0.0, 0.0, 0.0], SOCM = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 968.0967681711597, 0.0, 0.0, 0.0], GEN = [-0.0 -0.0 -0.0 -0.0; 0.0 0.0 0.0 -0.0; 906.1426712117587 1510.2377853529313 906.1426712117587 1510.2377853529313], REn = [0.0 -0.0 0.0 0.0; 281.3046591682413 0.0 0.0 412.8650565], CHARGE = [0.0 0.0 -0.0 0.0; 0.0 0.0 -0.0 0.0; … ; 0.0 0.0 -0.0 0.0; 0.0 0.0 -0.0 0.0], DISCHARGE = [0.0 -0.0 0.0 -0.0; 0.0 0.0 0.0 -0.0; … ; 0.0 -0.0 0.0 -0.0; 0.0 -0.0 0.0 -0.0], SOC = [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; … ; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0], COMMIT = [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 1.0 1.0 1.0 1.0], START = [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0], SHUT = [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.

In [4]:
sol.SOCM

14-element Vector{Float64}:
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
 968.0967681711597
   0.0
   0.0
   0.0

In [5]:
sol.CHARGE

14×4 Matrix{Float64}:
 0.0    0.0    -0.0    0.0
 0.0    0.0    -0.0    0.0
 0.0    0.0    -0.0    0.0
 0.0    0.0    -0.0    0.0
 0.0    0.0    -0.0    0.0
 0.0    0.0    -0.0    0.0
 0.0    0.0    -0.0    0.0
 0.0    0.0    -0.0    0.0
 0.0    0.0    -0.0    0.0
 0.0   -0.0    -0.0    0.0
 0.0  353.76  526.328  0.0
 0.0    0.0    -0.0    0.0
 0.0    0.0    -0.0    0.0
 0.0    0.0    -0.0    0.0

In [6]:
sol.DISCHARGE

14×4 Matrix{Float64}:
 0.0  -0.0  0.0   -0.0
 0.0   0.0  0.0   -0.0
 0.0   0.0  0.0   -0.0
 0.0  -0.0  0.0   -0.0
 0.0  -0.0  0.0   -0.0
 0.0  -0.0  0.0   -0.0
 0.0  -0.0  0.0   -0.0
 0.0  -0.0  0.0   -0.0
 0.0  -0.0  0.0   -0.0
 0.0   0.0  0.0   -0.0
 0.0   0.0  0.0  266.227
 0.0  -0.0  0.0   -0.0
 0.0  -0.0  0.0   -0.0
 0.0  -0.0  0.0   -0.0

In [7]:
sol.SOC

14×4 Matrix{Float64}:
   0.0      0.0      0.0      0.0
   0.0      0.0      0.0      0.0
   0.0      0.0      0.0      0.0
   0.0      0.0      0.0      0.0
   0.0      0.0      0.0      0.0
   0.0      0.0      0.0      0.0
   0.0      0.0      0.0      0.0
   0.0      0.0      0.0      0.0
   0.0      0.0      0.0      0.0
   0.0      0.0     -0.0      0.0
 484.048  678.616  968.097  484.048
   0.0      0.0      0.0      0.0
   0.0      0.0      0.0      0.0
   0.0      0.0      0.0      0.0

In [8]:
sol.CAPS

14-element Vector{Float64}:
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
  -0.0
 526.3283763017587
   0.0
   0.0
   0.0

## 2. Sensitivity Analysis of Storage Efficiencies

In [9]:
storageinfo = CSV.read(joinpath("data","cleaned","storageinfo.csv"), DataFrame)
gbgeneratorinfo = CSV.read(joinpath("data","cleaned","gbgeneratorinfo.csv"), DataFrame)
gbvreinfo = CSV.read(joinpath("data","cleaned","gbvreinfo.csv"), DataFrame)
gbhalfhourlydemand = CSV.read(joinpath("data","cleaned","gbhalfhourlydemand.csv"), DataFrame)
gbhalfhourlyvrecf = CSV.read(joinpath("data","cleaned","gbhalfhourlyvrecf.csv"), DataFrame)
gbhalfhourlydemand = gbhalfhourlydemand[1:48,:]
gbhalfhourlydemand[!,:load] = map(x->x-gbgeneratorinfo[gbgeneratorinfo.name.=="Nuclear",:installedMW][1],gbhalfhourlydemand[:,:load])
gbhalfl=hourlyvrecf = gbhalfhourlyvrecf[1:48,:];
gbhalfhourlydemand


Row,hh,load
,Int64,Int64
1,1,23607
2,2,24252
3,3,24565
4,4,23498
5,5,22784
6,6,22292
7,7,21365
8,8,20483
9,9,20590


In [10]:
# storageinfo: 
# id,storageMethod,name,installedMW,dischargeCost,chargeCost,powerCost,energyCost,chargeEff,dischargeEff,eff

# gbgeneratorinfo:
# id,name,installedMW,minPrate,fixedCost,varCost,startCostperMW,rampupRate,rampdownRate,minupTime,mindownTime,af

# gbvreinfo:
# id,name,installedMW,fixedCost,varCost

# gbhalfhourlydemand:
# hh,load

# gbhalfhourlyvrecf:
# id,name,installedMW,fixedCost,varCost

function GESP(storageinfo, gbgeneratorinfo, gbvreinfo, gbhalfhourlydemand, gbhalfhourlyvrecf; Budget=1e9, VoLL=30000, bigM = 10000, mip_gap=0.01)

    # SETS
    G = gbgeneratorinfo.id
    G = G[1:end-1]
    G_fixed = gbgeneratorinfo[(gbgeneratorinfo.name .== "Coal"),:id]
    R = gbvreinfo.id
    S = storageinfo.id
    T = gbhalfhourlydemand.hh
    T1 = T[2:end]

    # INIT
    model = Model()
    set_optimizer(model, HiGHS.Optimizer)
    set_optimizer_attribute(model, "mip_rel_gap", mip_gap)

    # VARIABLES
    @variables(model, begin
        CAPG[G] >= 0
        CAPG_COMMIT[G,T] >= 0
        CAPG_COMMIT2[G,T] >= 0
        CAPR[R] >= 0
        CAPS[S] >= 0
        SOCM[S] >= 0
        GEN[G,T] >= 0
        REN[R,T] >= 0
        CHARGE[S,T] >= 0
        CHARGE_STATE[S,T], Bin
        DISCHARGE[S,T] >= 0
        DISCHARGE_STATE[S,T], Bin
        SOC[S,T] >= 0
        COMMIT[G,T], Bin
        START[G,T], Bin
        SHUT[G,T], Bin
        NSE[T] >= 0
    end)

    # INVESTMENT COSTS
    @expression(model, GeneratorIC, sum(gbgeneratorinfo[gbgeneratorinfo.id .== g, :fixedCost][1] * (CAPG[g] - gbgeneratorinfo[gbgeneratorinfo.id.==g,:installedMW][1]) for g in G))
    @expression(model, VreIC, sum(gbvreinfo[gbvreinfo.id .== r, :fixedCost][1] * (CAPR[r] - gbvreinfo[gbvreinfo.id.==r,:installedMW][1]) for r in R))
    @expression(model, StoragePowIC, sum(storageinfo[storageinfo.id .== s, :powerCost][1] * CAPS[s] for s in S))
    @expression(model, StorageEnIC, sum(storageinfo[storageinfo.id .== s, :energyCost][1] * SOCM[s] for s in S))
    @expression(model, TIC, GeneratorIC + VreIC + StoragePowIC + StorageEnIC)
    @constraint(model, BudgetConstraint, TIC <= Budget)

    # OPERATION COSTS
    @expression(model, GeneratorOC, 
        sum(0.5 * gbgeneratorinfo[gbgeneratorinfo.id .== g, :varCost][1] * GEN[g,t] for g in G for t in T))
    @expression(model, VreOC, 
        sum(0.5 * gbvreinfo[gbvreinfo.id .== r, :varCost][1] * REN[r,t] for r in R for t in T))
    @expression(model, StartupOC, 
        sum(gbgeneratorinfo[gbgeneratorinfo.id .== g, :startCostperMW][1] * START[g,t] for g in G for t in T))
    @expression(model, NSE_C, 
        sum(VoLL * NSE[t] for t in T))
    @expression(model, TOC, GeneratorOC + VreOC + StartupOC + NSE_C)

    # OBJECTIVE FUNCTION
    @objective(model, Min, TOC)

    # POWER BALANCE CONSTRAINT
    @constraint(model, PowerBalanceC[t in T], sum(GEN[g,t] for g in G) + sum(REN[r,t] for r in R) + sum(DISCHARGE[s,t] for s in S) - sum(CHARGE[s,t] for s in S) + NSE[t] == gbhalfhourlydemand[gbhalfhourlydemand.hh.==t,:load][1])

    # Generator GENERATION LIMIT CONSTRAINT
    @constraint(model, LessThanMC[g in G, t in T], CAPG_COMMIT[g,t] <= bigM * COMMIT[g,t])
    @constraint(model, LessThanCapC[g in G, t in T], CAPG_COMMIT[g,t] <= CAPG[g])
    @constraint(model, AuxIsRealC[g in G, t in T], CAPG_COMMIT[g,t] >= CAPG[g] - (1- COMMIT[g,t]) * bigM)
    @constraint(model, GeneratorLimC[g in G, t in T], GEN[g,t] <= CAPG_COMMIT[g,t])
    @constraint(model, GeneratorMinLimC[g in G, t in T], GEN[g,t] >= gbgeneratorinfo[gbgeneratorinfo.id.==g,:minPrate][1] * CAPG_COMMIT[g,t])

    # existing capacity constraints
    @constraint(model, GeneratorEC[g in G], CAPG[g] >= gbgeneratorinfo[gbgeneratorinfo.id.==g,:installedMW][1])
    @constraint(model, VreEC[r in R], CAPR[r] >= gbvreinfo[gbvreinfo.id.==r,:installedMW][1])

    # cannot build new nuclear and coal constraint
    @constraint(model, GeneratorFixed[g in G_fixed], CAPG[g] <= gbgeneratorinfo[gbgeneratorinfo.id.==g,:installedMW][1])

    # START CONSTRAINT
    @constraint(model, StartC[g in G, t in T], 
        COMMIT[g,t] >= sum(START[g,i] for i in max(1,t-gbgeneratorinfo[gbgeneratorinfo.id.==g,:minupTime][1]):t))

    # SHUT CONSTRAINT
    @constraint(model, ShutC[g in G, t in T], 
        1 - COMMIT[g,t] >= sum(SHUT[g,i] for i in max(1,t-gbgeneratorinfo[gbgeneratorinfo.id.==g,:mindownTime][1]):t))

    # COMMIT CONSTRAINT
    @constraint(model, CommitC[g in G, t in T1], COMMIT[g,t] - COMMIT[g,t-1] == START[g,t] - SHUT[g,t])

    # Generator RAMP UP CONSTRAINT
    @constraint(model, TherRampUpC[g in G, t in T1], 
        GEN[g,t] - GEN[g,t-1] <= gbgeneratorinfo[gbgeneratorinfo.id.==g,:rampupRate][1] * CAPG[g])
    
    # Generator RAMP DOWN CONSTRAINT
    @constraint(model, TherRampDnC[g in G, t in T1], 
        GEN[g,t-1] - GEN[g,t] <= gbgeneratorinfo[gbgeneratorinfo.id.==g,:rampdownRate][1] * CAPG[g])

    # VRE LIMIT CONSTRAINT
    @constraint(model, VreC[r in R, t in T], 
        REN[r,t] <= CAPR[r] * gbhalfhourlyvrecf[(gbhalfhourlyvrecf.vre_id.==r) .& (gbhalfhourlyvrecf.hh.==t), :cf][1])
    
    # CHARGE LIMIT CONSTRAINT
    @constraint(model, ChargeC[s in S, t in T], CHARGE[s,t] <= CAPS[s])
    @constraint(model, ChargeIni[s in S], CHARGE[s,1] == 0)

    # DISCHARGE LIMIT CONSTRAINT
    @constraint(model, DischargeC[s in S, t in T], DISCHARGE[s,t] <= CAPS[s])
    @constraint(model, DischargeIni[s in S], DISCHARGE[s,1] == 0)

    # SOC LIMIT CONSTRAINT
    @constraint(model, SocLimitC[s in S, t in T], SOC[s,t] <= SOCM[s])

    # SOC UPDATE CONSTRAINT
    @constraint(model, SocUpdateC[s in S, t in T1],
        SOC[s, t] == SOC[s,t-1] + CHARGE[s,t] - DISCHARGE[s,t])
    
    # SOC INI CONSTRAINT
    @constraint(model, SocIniC[s in S], SOC[s,1] == 0.5 * SOCM[s])

    # SOC END CONSTRAINT
    @constraint(model, SocEndC[s in S], SOC[s,length(T)] == 0.5 * SOCM[s])

    # avoid commit before start
    @constraint(model, StartBefCommitC[g in G, t in T], COMMIT[g,t] <= sum(START[g,i] for i in minimum(T):t))

    # SOLVE GESP
    optimize!(model)

    return(
        CAPG = value.(CAPG).data,
        CAPG_COMMIT = value.(CAPG_COMMIT).data,
        CAPR = value.(CAPR).data,
        CAPS = value.(CAPS).data,
        SOCM = value.(SOCM).data,
        GEN = value.(GEN).data,
        REN = value.(REN).data,
        CHARGE = value.(CHARGE).data,
        DISCHARGE = value.(DISCHARGE).data,
        SOC = value.(SOC).data,
        COMMIT = value.(COMMIT).data,
        START = value.(START).data,
        SHUT = value.(SHUT).data,
        NSE = value.(NSE).data,
        GeneratorIC = value(GeneratorIC),
        StoragePowIC = value(StoragePowIC),
        StorageEnIC = value(StorageEnIC),
        GeneratorOC = value(GeneratorOC),
        VreOC = value(VreOC),
        StartupOC = value(StartupOC),
        NSE_C = value(NSE_C),
        TIC = value(TIC),
        TOC = value(TOC),
        OV = objective_value(model)
    )
end

GESP (generic function with 4 methods)

In [11]:
sol = GESP(storageinfo, gbgeneratorinfo, gbvreinfo, gbhalfhourlydemand, gbhalfhourlyvrecf; Budget= 1000000, VoLL=100000, bigM = 100000, mip_gap=0.01)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
5349 rows, 3348 cols, 22044 nonzeros
4786 rows, 2991 cols, 19305 nonzeros

Solving MIP model with:
   4786 rows
   2991 cols (454 binary, 0 integer, 1 implied int., 2536 continuous)
   19305 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   133512.961956   inf                  inf        0      0      0         0     0.1s
 R       0       0         0   0.00%   7535638.989574  412008135.6072    98.17%        0      0      0      3412     0.4s
 C       0       0         0   0.00%   8334735.945573  411926991.6072    97.98%     1367     47      0      4612     0.7s
 L       0       0         0   0.00%   8883232.988462  10639474.62063    16.51%     2828    266      

(CAPG = [1409.0, 28785.0, 3381.0, 1466.0, 2580.0], CAPG_COMMIT = [1409.0 1409.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 1466.0 1466.0 … -2.2737367544323206e-13 0.0; 0.0 0.0 … 0.0 0.0], CAPR = [4881.0, 11484.0, 14362.0], CAPS = [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, 1740.0796580310878, -0.0, -0.0, -0.0], SOCM = [-0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 10860.336658031089, -0.0, -0.0, -0.0], GEN = [563.5999999999985 563.6 … 0.0 0.0; 4.547473508864641e-13 -0.0 … 0.0 0.0; … ; 1466.0 1466.0 … -2.2737367544323206e-13 -0.0; -0.0 0.0 … -0.0 0.0], REN = [0.0 0.0 … 0.0 0.0; 7840.1268 7840.1268 … 9392.54785803109 8745.54785803109; 10813.149800000001 10813.149800000001 … 13843.531799999999 13843.531799999999], CHARGE = [0.0 -0.0 … -0.0 -0.0; 0.0 0.0 … -0.0 0.0; … ; 0.0 0.0 … -0.0 -0.0; 0.0 0.0 … -0.0 -0.0], DISCHARGE = [0.0 0.0 … 0.0 0.0; 0.0 -0.0 … -0.0 -0.0; … ; 0.0 -0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0], SOC = [0.0 -0.0 … -0.0 0.0; 0.0 -0.0 … -0.0 0.0; … ; 0.0 -0.0 … -0